In [169]:
import pandas as pd
from icecream import ic
%matplotlib inline
from pathlib import Path
from src.data.extract import extract, processing
import numpy as np
from decimal import Decimal, ROUND_HALF_UP
import pygwalker as pyg
import openpyxl
import duckdb
from src.tools.data_features import day_differnence,get_df_last_week,conver_summ_to_m,custom_agg,date_views
from src.tools.data_implemetation import create_xlsx_process,create_csv_process
from pathlib import Path
import json
with open(r'C:\Users\Лисицынский Никита\alalytics\projects\analyst_platform\glossary.json', 'r', encoding='utf-8') as json_file:
    loaded_dict = json.load(json_file)



In [170]:
exract_data = extract(zip_file_path=r"C:\Users\Лисицынский Никита\Downloads\drive-download-20231129T112855Z-001.zip")


In [171]:
link_current = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTlFWb1KJzGk0DaR88jKQPujsSCmLVlEm7oDfaY5CrGowa-YecDAfc5F__JmCbiTGiGDqiHJP5_QecM/pub?gid=0&single=true&output=csv'
df = pd.read_csv(link_current)


In [172]:
df_all = pd.concat((exract_data,df))

In [173]:
df_raw = processing(df_all)


In [6]:
df_raw['stage_name'].value_counts()

stage_name
Неопределенность длительная    40557
В работе                       21945
Интерес                        13402
Неопределенность короткая      12457
Отказ                          10338
Договорная стадия                695
Другая заявка                    464
Договор подписан                 389
Name: count, dtype: int64

In [174]:
check_columns = ['Год создания заявки','Номер календарной создания заявки',	'Код заявки'	,'Клиент'	,'Дата создания завявки'	,'Общая площадь'	,'Общая сумма','Первый контакт','Менеджер заявки','life_satage_number','stage_name','datetime_order_create','datetime_order_state']
last_raw = df_raw[check_columns].tail()
last_raw['Дата создания завявки'].tail(1)

34778    21.12.2023 10:30:01
Name: Дата создания завявки, dtype: object

In [22]:
# src_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQcTHahBvqHzqGdj_Vd-MZy87p7J5qwYKHhQXOvZyQ4b621b14X4jNdV33zqr2lkpMYt4wCfmtyK5Ha/pub?gid=2140019342&single=true&output=csv'
# df_raw = pd.read_csv(src_link)

In [175]:
df_raw = df_raw.rename(columns=loaded_dict)
df_raw.columns
df_raw= df_raw.drop(columns=['Unnamed: 28','Unnamed: 0','Unnamed: 29','Unnamed: 30','Unnamed: 11'],errors='ignore').rename(columns=({'Этап жиз. цикла':'stage_epoches'}))

In [20]:
df_raw.columns
# 'Unnamed: 29','Unnamed: 30','Unnamed: 11'

Index(['order_create_year_create', 'order_date_week_number_create', 'order_id',
       'customer', 'date_order_create', 'square_m2', 'amount_sum',
       'order_supplier', 'order_manager', 'AO', 'AC',
       'order_status_year_create', 'order_date_week_number_state',
       'date_time_state', 'state_type', 'order_state', 'order_state_check',
       'last_first_sate', 'variaty', 'contact_initiator', 'contact_person',
       'object_delivery_date', 'signing_date', 'note', 'note_author',
       'stage_epoches', 'order_link', ' ', 'order_state_time',
       'life_satage_number', 'stage_name', 'datetime_order_create',
       'datetime_order_state'],
      dtype='object')

# collect
- [ ] get [['']]

In [25]:
# df_raw.query('')

In [176]:
# null : 'order_link',

state = [
    'order_id','order_manager',
    'order_state','order_state_check','life_satage_number',
    'stage_name',
    'contact_person',
    'signing_date','object_delivery_date',
    'note','note_author',

    'amount_sum','square_m2','contact_initiator','order_supplier',

    'order_status_year_create','order_date_week_number_state','datetime_order_state']
create = [
    'order_id',
    'order_create_year_create','order_date_week_number_create','date_order_create']

In [177]:
df_create = df_raw[create]
df_states = df_raw[state]

distrib = [
    'order_id','order_manager',
    'order_state','order_state_check','life_satage_number','stage_name','order_supplier',
    'amount_sum','square_m2',
    'order_status_year_create','order_date_week_number_state','datetime_order_state'
]
df_distrib = df_states[distrib]

others = [
    'order_id','contact_person','object_delivery_date',
    'note','note_author','contact_initiator',
]
df_others = df_states[others]




In [28]:
# df_create = df_create.copy()
# df_create.groupby('order_id').agg(
#     'nunique').sum()


# main [ 
- df_distrib
- df_create ]

- [x] df['year_week']
- [ ] df['_index_']

In [178]:
# df['date_create_m'] = pd.to_datetime(df['date_order_create'],dayfirst=True,).dt.month
# df['date_create_y'] = pd.to_datetime(df['date_order_create'],dayfirst=True,).dt.year

df_raw['year_week'] = df_raw['order_create_year_create'].astype(str) + '-'+ df_raw['order_date_week_number_create'].astype(str)
df_raw["_index_"] = df_raw.index
df_process = df_raw.copy()
df_raw['year_week']


0        2019-46
1        2019-46
2        2019-46
3        2019-46
4        2019-46
          ...   
34774    2023-51
34775    2023-51
34776    2023-51
34777    2023-51
34778    2023-51
Name: year_week, Length: 100247, dtype: object

In [179]:
# Apply the function to your column
df = df_raw.copy()
def clean_and_convert(value):
    try:
        # Try converting to int
        result = int(value)
    except ValueError:
        # If it's not a valid int, apply transformation
        result = float(str(value).replace('\xa0', '').replace(',', '.'))

    return result


df['amount_sum'] = df['amount_sum'].apply(clean_and_convert)
df['sum_in_mln'] = df.amount_sum / 1_000_000
df['square_m2'] = (df['square_m2'].apply(clean_and_convert)).round(2)
df['customer'] = df['customer'].str.replace('[()]', ' ', regex=True).str.strip()
# df['customer'] = df['customer'].str.replace('Трест РосСЭМ', 'Трест РосСЭМ АтомСтройЭкспорт')
df['customer'] = df['customer'].astype(str)
mask = df['customer'].str.contains('Трест РосСЭМ АтомСтройЭкспорт', na=False)
df.loc[mask, 'customer'] = 'Трест РосСЭМ'
df_process = df.copy()


df_process

In [180]:
# df_process.rename(columns=loaded_dict)
df_raw_data = df_process.rename(columns=loaded_dict)
df_raw_data

,order_create_year_create,order_date_week_number_create,order_id,customer,date_order_create,square_m2,amount_sum,order_supplier,order_manager,AO,...,order_link,,order_state_time,life_satage_number,stage_name,datetime_order_create,datetime_order_state,year_week,_index_,sum_in_mln
0,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,010,Интерес,2019-11-11 10:26:03,2019-11-11 07:26:03,2019-46,0,51.3036
1,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-13 13:58:53,2019-46,1,51.3036
2,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 07:56:37,2019-46,2,51.3036
3,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 10:15:09,2019-46,3,51.3036
4,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,NaT,2019-46,4,51.3036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34774,2023,51,15647,Резервуарный завод ВЕССЕЛ,20.12.2023 13:59:37,238.00,0.0,Эл. почта,Демчишин Павел Сергеевич,2,...,NaN,NaN,1564721.12.2023 07:15:22,060,Отказ,2023-12-20 13:59:37,2023-12-21 07:15:22,2023-51,34774,0.0000
34775,2023,51,15648,ДАКС-ИНЖИНИРИНГ,20.12.2023 15:53:13,14.74,0.0,Эл. почта,Ратникова Анна Владимировна,1,...,NaN,NaN,1564820.12.2023 12:53:13,010,Интерес,2023-12-20 15:53:13,2023-12-20 12:53:13,2023-51,34775,0.0000
34776,2023,51,15649,Лимак-констракшн,20.12.2023 16:13:56,90.80,0.0,Эл. почта,Васильев Семен Вячеславович,1,...,NaN,NaN,1564920.12.2023 13:13:56,010,Интерес,2023-12-20 16:13:56,2023-12-20 13:13:56,2023-51,34776,0.0000
34777,2023,51,15650,Всероссийский дважды ордена Трудового Красного Знамени Теплотехнический научно-исследовательский институт,21.12.2023 09:29:25,0.00,0.0,Эл. почта,Ратникова Анна Владимировна,1,...,NaN,NaN,1565021.12.2023 06:29:25,010,Интерес,2023-12-21 09:29:25,2023-12-21 06:29:25,2023-51,34777,0.0000


In [32]:

create_csv_process(
    path_intro=Path(r"G:\Мой диск\analyze\ORDERS\DataMart\eng"),
    TABLES={'orders_raw':df_raw_data}
)
create_csv_process(
    path_intro=Path(r"G:\Мой диск\analyze\ORDERS\DataMart\рус"),
    TABLES={'orders_raw':df_raw_data}
)


In [181]:
df_raw_data.head()

,order_create_year_create,order_date_week_number_create,order_id,customer,date_order_create,square_m2,amount_sum,order_supplier,order_manager,AO,...,order_link,,order_state_time,life_satage_number,stage_name,datetime_order_create,datetime_order_state,year_week,_index_,sum_in_mln
0,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.0,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,010,Интерес,2019-11-11 10:26:03,2019-11-11 07:26:03,2019-46,0,51.3036
1,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.0,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-13 13:58:53,2019-46,1,51.3036
2,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.0,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 07:56:37,2019-46,2,51.3036
3,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.0,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 10:15:09,2019-46,3,51.3036
4,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.0,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,NaT,2019-46,4,51.3036


## 


In [182]:


df = df_process.copy()

analyse_states_columns = ['order_id','order_state','order_manager','life_satage_number','stage_name','sum_in_mln','square_m2',
 'order_status_year_create','order_date_week_number_state','datetime_order_state','year_week','_index_',]
others_states_columns = ['order_id','order_manager','order_state_check','order_supplier',
 'order_status_year_create','order_date_week_number_state','datetime_order_state','year_week','_index_']

df_analyse = df[analyse_states_columns]
df_analyse.sample()

,order_id,order_state,order_manager,life_satage_number,stage_name,sum_in_mln,square_m2,order_status_year_create,order_date_week_number_state,datetime_order_state,year_week,_index_
5156,7789,Спящий,Хомутский Михаил Александрович,050,Неопределенность длительная,3.686,235.0,2018,29,NaT,2018-26,5156


In [15]:
df_process.columns

Index(['order_create_year_create', 'order_date_week_number_create', 'order_id',
       'customer', 'date_order_create', 'square_m2', 'amount_sum',
       'order_supplier', 'order_manager', 'AO', 'AC',
       'order_status_year_create', 'order_date_week_number_state',
       'date_time_state', 'state_type', 'order_state', 'order_state_check',
       'last_first_sate', 'variaty', 'contact_initiator', 'contact_person',
       'object_delivery_date', 'signing_date', 'note', 'note_author',
       'stage_epoches', 'order_link', ' ', 'order_state_time',
       'life_satage_number', 'stage_name', 'datetime_order_create',
       'datetime_order_state', 'year_week', '_index_', 'sum_in_mln'],
      dtype='object')

## ordirs_input()

In [34]:
df =  df_process[[
    'order_id','order_manager','stage_name','order_supplier','amount_sum','square_m2',
  'date_order_create','datetime_order_state','year_week','_index_',]].copy()
def custom_last_value(x):
    last_valid_index = x.last_valid_index()
    return x[last_valid_index] if last_valid_index is not None and x[last_valid_index] != 0 else None
df['have_signed'] = df.groupby('order_id')['stage_name'].transform(lambda x: (x == 'Договор подписан').any())

dates_significant = df[df['stage_name'] == 'Договор подписан'].groupby('order_id')[['order_id','datetime_order_state']].agg(
    datetime_order_state=("datetime_order_state","min")).rename(columns=({'datetime_order_state':'date_signed'})).reindex()


df = df.sort_values(by='year_week').groupby('order_id') \
    .agg(
        create_date = ('date_order_create','min'),
        order_manager_last = ('order_manager','last'),
        last_datetime_order_state =('datetime_order_state','last'),
        last_stage_name = ('stage_name','last'),
        last_amount_sum=('amount_sum',custom_last_value),

        last_square_m2 = ('square_m2',custom_last_value),
        last_index_ = ('_index_','last'),
        last_year_week= ('year_week','max'),
        have_signed = ('have_signed','last'),
        
         )
df[['last_amount_sum','last_square_m2']] = df[['last_amount_sum','last_square_m2']].fillna(0)
df['last_sum_in_mln'] =  round(df['last_amount_sum'] / 1_000_000,3)



df = df.merge(dates_significant, on='order_id',how='outer')
df['date_create_m'] = pd.to_datetime(df['create_date'],dayfirst=True,).dt.month
df['date_create_y'] = pd.to_datetime(df['create_date'],dayfirst=True,).dt.year
df['date_last_state_m'] = pd.to_datetime(df['last_datetime_order_state'],dayfirst=True,).dt.month

df['date_last_state_y'] = pd.to_datetime(df['last_datetime_order_state'],dayfirst=True,).dt.year

df_state_rows = df.reset_index()
df_state_rows.sample(4)


,order_id,create_date,order_manager_last,last_datetime_order_state,last_stage_name,last_amount_sum,last_square_m2,last_index_,last_year_week,have_signed,last_sum_in_mln,date_signed,date_create_m,date_create_y,date_last_state_m,date_last_state_y
6970,13643,14.12.2022 11:28:21,Васильев Семен Вячеславович,2022-12-14 08:28:21,Интерес,0.0,14.0,9259,2022-50,False,0.000,NaT,12,2022,12,2022
1676,8349,2019-01-28 15:28:05,Самохина Анастасия,2019-01-28 12:28:05,Интерес,37050000.0,1173.0,4544,2019-5,False,37.050,NaT,1,2019,1,2019
2954,9627,2020-01-30 10:03:00,Полякова Анна,2020-01-30 07:03:00,Интерес,0.0,212.0,1049,2020-5,False,0.000,NaT,1,2020,1,2020
4931,11604,2021-07-21 10:08:39,Шуляк Мария,2021-08-30 11:08:44,В работе,33556000.0,472.0,3944,2021-29,False,33.556,NaT,7,2021,8,2021


In [35]:


df_state_rows = df.copy()
create_csv_process(
    path_intro=Path(r"G:\Мой диск\analyze\ORDERS\orders_analyst_test"),
    TABLES={'заявки и стостояния':df}
)


## Second_state_row


In [84]:
df_state_rows.last_stage_name.value_counts()
df = df_state_rows[['order_id' ,'date_create_y','date_create_m','date_last_state_y','date_last_state_m']].copy()
df.query()       

,order_id,date_create_y,date_create_m,date_last_state_y,date_last_state_m
0,6673,2017,1,2017,2
1,6674,2017,1,2017,5
2,6675,2017,1,2017,1
3,6676,2017,1,2017,1
4,6677,2017,1,2017,1
...,...,...,...,...,...
8975,15648,2023,12,2023,12
8976,15649,2023,12,2023,12
8977,15650,2023,12,2023,12
8978,15651,2023,12,2023,12


df_state_rows

In [49]:
df_state_rows =df_state_rows.reset_index()

In [31]:
list_another_order = df_state_rows.query("last_stage_name == 'Другая заявка'").order_id.to_list()

In [51]:
df = df_state_rows.copy()

def grouped_year_month(
      df_state_rows:pd.DataFrame,
      date_create_y:str='date_create_y',
      date_create_m:str='date_create_m'
      )->pd.DataFrame:
   df = df_state_rows.groupby([date_create_y,date_create_m]) \
    .agg(
        order_id=('order_id','count'),
        sum_last_square_m2 = ('last_square_m2','sum'),
        sum_last_sum_in_mln = ('last_sum_in_mln','sum'),
        count_unuque_employers = ('order_manager_last','nunique'),
        emploers_in = ('order_manager_last','unique'),
    ).reset_index()
   return df
df_agg_orders_month = grouped_year_month(df_state_rows=df_state_rows)

df_agg_orders_month_signed = grouped_year_month(df_state_rows=df_state_rows[df_state_rows['have_signed'] == True])

create_csv_process(
    path_intro=Path(r"G:\Мой диск\analyze\ORDERS\orders_analyst_test"),
    TABLES={'заявки_месяцы_':df_agg_orders_month}
)
create_csv_process(
    path_intro=Path(r"G:\Мой диск\analyze\ORDERS\orders_analyst_test"),
    TABLES={'заявки_месяцы_подписан':df_agg_orders_month_signed}
)


In [45]:
# df_state_rows.groupby(['date_create_y','date_create_m']) # dates_significant
# df_state_rows.query('have_signed == True')
# df_state_rows

In [52]:

df_agg_orders_month_signed = grouped_year_month(
    df_state_rows=
        date_views(
            df=df_state_rows.query('have_signed == True'),
                date_column='date_signed'),
    date_create_y =   'date_signed_year',
    date_create_m= 'date_signed_week'               
                  
)
df_agg_orders_month_signed


c:\Users\Лисицынский Никита\alalytics\projects\orders_distribution\src\tools\data_features.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{date_column}_date'] = df[date_column].dt.date
c:\Users\Лисицынский Никита\alalytics\projects\orders_distribution\src\tools\data_features.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{date_column}_week'] = df[date_column].dt.isocalendar().week
c:\Users\Лисицынский Никита\alalytics\projects\orders_distribution\src\tools\data_features.p

,date_signed_year,date_signed_week,order_id,sum_last_square_m2,sum_last_sum_in_mln,count_unuque_employers,emploers_in
0,2017,8,1,0.00,2.829,1,[Дубовой Сергей]
1,2017,12,1,1032.00,18.810,1,[Дубовой Сергей]
2,2017,17,1,905.00,17.250,1,[Самохина Анастасия]
3,2017,18,1,144.00,2.848,1,[Самохина Анастасия]
4,2017,20,1,6.00,0.115,1,[Дубовой Сергей]
...,...,...,...,...,...,...,...
170,2023,46,2,2503.11,121.733,2,"[Байкалов Александр Александрович, Шуляк Мария]"
171,2023,47,1,5395.30,413.541,1,[Дубовой Сергей]
172,2023,48,1,206.60,20.381,1,[Самохина Анастасия]
173,2023,49,3,4710.01,351.566,1,[Шуляк Мария]


## signed analyse

In [53]:
df = df_agg_orders_month_signed \
    .explode('emploers_in').groupby('date_signed_year') \
    .agg(
        order_id_count = ("order_id",'count'),
        sum_last_square_m2 = ('sum_last_square_m2','sum'),
        sum_last_sum_in_mln = ('sum_last_sum_in_mln','sum'),
        count_unuque_employers = ('count_unuque_employers','nunique'),
        emploers_in = ('emploers_in','unique')
    )
df['sum_last_sum_in_mln'] = df.sum_last_sum_in_mln.astype('int')
df['sum_last_square_m2'] = df.sum_last_square_m2.astype('int')
df.reset_index()

,date_signed_year,order_id_count,sum_last_square_m2,sum_last_sum_in_mln,count_unuque_employers,emploers_in
0,2017,26,31433,786,3,"[Дубовой Сергей, Самохина Анастасия, Шуляк Мар..."
1,2018,46,106650,4028,4,"[Шуляк Мария, Полякова Анна, Дубовой Сергей, С..."
2,2019,56,61621,2036,4,"[Бевзюк Мария, Дубовой Сергей, Самохина Анаста..."
3,2020,23,15206,475,2,"[Дубовой Сергей, Полякова Анна, Евгений Ерофее..."
4,2021,23,75857,4143,3,"[Шуляк Мария, Дубовой Сергей, Чикова Ираида, П..."
5,2022,32,44426,2966,2,"[Самохина Анастасия, Шуляк Мария, Дубовой Серг..."
6,2023,34,78575,6109,2,"[Шуляк Мария, Васильев Семен Вячеславович, Дуб..."


## df_signing contracts

In [37]:
df_analyse.query("order_id == 10512") # 14256 1310.46 5085
# df_analyse.query("order_id == '5085'")

,order_id,order_state,order_manager,life_satage_number,stage_name,sum_in_mln,square_m2,order_status_year_create,order_date_week_number_state,datetime_order_state,year_week,_index_
9510,10512,Интерес,Полякова Анна,010,Интерес,452.354787,2941.0,2020,39,2020-09-24 08:10:01,2020-39,9510
9511,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,39,2020-09-24 08:16:19,2020-39,9511
9512,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,40,NaT,2020-39,9512
9513,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,41,NaT,2020-39,9513
9514,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,42,2020-10-12 08:26:27,2020-39,9514
9515,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,42,2020-10-16 14:29:23,2020-39,9515
9516,10512,Уточнение ТЗ,Полякова Анна,020,Неопределенность короткая,452.354787,2941.0,2020,43,2020-10-20 09:29:05,2020-39,9516
9517,10512,Запрос КП,Полякова Анна,040,В работе,452.354787,2941.0,2020,44,2020-10-27 10:34:13,2020-39,9517
9518,10512,Запрос КП,Полякова Анна,040,В работе,452.354787,2941.0,2020,45,NaT,2020-39,9518
9519,10512,Запрос КП,Полякова Анна,040,В работе,452.354787,2941.0,2020,46,NaT,2020-39,9519


## analyse_1

In [183]:
dict_states_new = {
    'В работе' : "Принятых в работу: кол-во",
    'Договорная стадия':'Заявок на договорной стадии: кол-во',
    "Отказ":'Отказы: кол-во',
    'Приостановка (неопределенность длительная): кол-во':'Неопределенность длительная'
}

In [184]:
df=df_process.copy()
df

,order_create_year_create,order_date_week_number_create,order_id,customer,date_order_create,square_m2,amount_sum,order_supplier,order_manager,AO,...,order_link,,order_state_time,life_satage_number,stage_name,datetime_order_create,datetime_order_state,year_week,_index_,sum_in_mln
0,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,010,Интерес,2019-11-11 10:26:03,2019-11-11 07:26:03,2019-46,0,51.3036
1,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-13 13:58:53,2019-46,1,51.3036
2,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 07:56:37,2019-46,2,51.3036
3,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,2019-11-22 10:15:09,2019-46,3,51.3036
4,2019,46,9322,РДМ,2019-11-11 10:26:03,1100.00,51303600.0,Эл. почта,Бевзюк Мария,0,...,NaN,NaN,NaN,040,В работе,2019-11-11 10:26:03,NaT,2019-46,4,51.3036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34774,2023,51,15647,Резервуарный завод ВЕССЕЛ,20.12.2023 13:59:37,238.00,0.0,Эл. почта,Демчишин Павел Сергеевич,2,...,NaN,NaN,1564721.12.2023 07:15:22,060,Отказ,2023-12-20 13:59:37,2023-12-21 07:15:22,2023-51,34774,0.0000
34775,2023,51,15648,ДАКС-ИНЖИНИРИНГ,20.12.2023 15:53:13,14.74,0.0,Эл. почта,Ратникова Анна Владимировна,1,...,NaN,NaN,1564820.12.2023 12:53:13,010,Интерес,2023-12-20 15:53:13,2023-12-20 12:53:13,2023-51,34775,0.0000
34776,2023,51,15649,Лимак-констракшн,20.12.2023 16:13:56,90.80,0.0,Эл. почта,Васильев Семен Вячеславович,1,...,NaN,NaN,1564920.12.2023 13:13:56,010,Интерес,2023-12-20 16:13:56,2023-12-20 13:13:56,2023-51,34776,0.0000
34777,2023,51,15650,Всероссийский дважды ордена Трудового Красного Знамени Теплотехнический научно-исследовательский институт,21.12.2023 09:29:25,0.00,0.0,Эл. почта,Ратникова Анна Владимировна,1,...,NaN,NaN,1565021.12.2023 06:29:25,010,Интерес,2023-12-21 09:29:25,2023-12-21 06:29:25,2023-51,34777,0.0000


In [229]:
df=df_process.copy()
df[['last_amount_sum','last_square_m2']] = df[['amount_sum','square_m2']].fillna(0)
df['last_sum_in_mln'] =  round(df['last_amount_sum'] / 1_000_000,3)



df = df.merge(dates_significant, on='order_id',how='outer')
# df['date_create_m'] = pd.to_datetime(df['create_date'],dayfirst=True,).dt.month
# df['date_create_y'] = pd.to_datetime(df['create_date'],dayfirst=True,).dt.year
df['date_last_state_m'] = pd.to_datetime(df['datetime_order_state'],dayfirst=True,).dt.month

df['date_last_state_y'] = pd.to_datetime(df['datetime_order_state'],dayfirst=True,).dt.year
df
df = df.groupby(['order_id','date_last_state_y','date_last_state_m','stage_name']).agg('last').reset_index()

def create_month(month, year):
    return f'01.{month}.{year}'
df
df['date_last_state_m'] = df['date_last_state_m'].fillna(0).astype('int').astype('str').str.zfill(2)
df['date_last_state_y'] = df['date_last_state_y'].fillna(0).astype('int').astype('str')
df['new_date'] = df.apply(lambda row: create_month(row['date_last_state_m'], row['date_last_state_y']), axis=1)
df_val = df.copy()
df = df_val[['order_id','date_last_state_y','date_last_state_m','stage_name','last_sum_in_mln','new_date']]
df_overview = (df.groupby(['date_last_state_y','date_last_state_m','stage_name'])
 [['order_id','last_sum_in_mln','new_date']]
 .agg({'order_id':'count',
       'last_sum_in_mln':'sum',
       'new_date':'last'}).reset_index()
.query("date_last_state_y == '2023'")
.query("date_last_state_m > '05'")
)


# df['last_stage_name'] = 
df_overview['new_names'] = df_overview['stage_name'].replace(dict_states_new)
list_val = dict_states_new.values()
# df_overview
df_overview[['new_date','new_names','order_id']].query("new_names in @list_val")



,new_date,new_names,order_id
508,01.06.2023,Принятых в работу: кол-во,107
510,01.06.2023,Заявок на договорной стадии: кол-во,8
513,01.06.2023,Неопределенность длительная,112
515,01.06.2023,Отказы: кол-во,119
516,01.07.2023,Принятых в работу: кол-во,95
518,01.07.2023,Заявок на договорной стадии: кол-во,11
521,01.07.2023,Неопределенность длительная,82
523,01.07.2023,Отказы: кол-во,138
524,01.08.2023,Принятых в работу: кол-во,119
526,01.08.2023,Заявок на договорной стадии: кол-во,20


In [256]:
# df_val.groupby(['order_create_year_create,order_date_week_number_create','order_id']).count()
df = df_val.copy()
# df['date_first_state_m'] = pd.to_datetime(df['date_order_create'],dayfirst=True,).dt.month
df['create_month'] = pd.to_datetime(df['date_order_create'],dayfirst=True,errors='coerce').dt.month.fillna(0).astype('int').astype('str').str.zfill(2)
df['new_date'] = df.apply(lambda row: create_month(row['create_month'], row['order_create_year_create']), axis=1)

df_month = df.query('create_month != "00"')
(df_month
 .query("date_last_state_y == '2023'")
.query("date_last_state_m > '05'")).groupby(['new_date'])['order_id'].count().reset_index()
# data = df_month


,new_date,order_id
0,01.01.2023,36
1,01.02.2023,37
2,01.03.2023,88
3,01.04.2023,86
4,01.05.2023,230
5,01.06.2023,678
6,01.07.2023,635
7,01.08.2023,832
8,01.09.2023,659
9,01.10.2023,620


In [294]:
(df_overview.query('new_names == "Заявок на договорной стадии: кол-во"')[['new_date','new_names','last_sum_in_mln']]
.assign(summ = (((df_overview['last_sum_in_mln']*1000).astype('uint64')*1_000).astype('str')))).drop(columns=['last_sum_in_mln']).to_clipboard()

In [125]:

# caledar_dict = {

# }

,order_id,create_date,order_manager_last,last_datetime_order_state,last_stage_name,last_amount_sum,last_square_m2,last_index_,last_year_week,have_signed,last_sum_in_mln,date_signed,date_create_m,date_create_y,date_last_state_m,date_last_state_y,new_date
0,6673,09.01.2017 15:19:31,Дубовой Сергей,2017-02-08 12:41:55,Неопределенность длительная,22440000.0,997.00,2470,2017-2,False,22.440,NaT,1,2017,02,2017,01.02.2017
1,6674,10.01.2017 10:02:42,Самохина Анастасия,2017-05-04 14:43:56,Договор подписан,2848500.0,144.00,7956,2017-2,True,2.848,2017-05-04 14:43:56,1,2017,05,2017,01.05.2017
2,6675,11.01.2017 10:32:46,Дубовой Сергей,2017-01-20 12:36:36,В работе,105860000.0,5581.00,941,2017-2,False,105.860,NaT,1,2017,01,2017,01.01.2017
3,6676,11.01.2017 11:27:05,Самохина Анастасия,2017-01-12 14:36:34,В работе,22935000.0,1092.00,944,2017-2,False,22.935,NaT,1,2017,01,2017,01.01.2017
4,6677,12.01.2017 15:26:05,Полякова Анна,2017-01-19 07:33:11,В работе,2007000.0,88.00,950,2017-2,False,2.007,NaT,1,2017,01,2017,01.01.2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8975,15648,20.12.2023 15:53:13,Ратникова Анна Владимировна,2023-12-20 12:53:13,Интерес,0.0,14.74,19823,2023-51,False,0.000,NaT,12,2023,12,2023,01.12.2023
8976,15649,20.12.2023 16:13:56,Васильев Семен Вячеславович,2023-12-20 13:13:56,Интерес,0.0,90.80,19824,2023-51,False,0.000,NaT,12,2023,12,2023,01.12.2023
8977,15650,21.12.2023 09:29:25,Ратникова Анна Владимировна,2023-12-21 06:29:25,Интерес,0.0,0.00,19825,2023-51,False,0.000,NaT,12,2023,12,2023,01.12.2023
8978,15651,21.12.2023 10:30:01,Ратникова Анна Владимировна,2023-12-21 07:30:01,Интерес,0.0,0.00,34778,2023-51,False,0.000,NaT,12,2023,12,2023,01.12.2023


In [107]:
df_last_statuses_month.tail()

,order_id,date_last_state_y,date_last_state_m,create_date,order_manager_last,last_datetime_order_state,last_stage_name,last_amount_sum,last_square_m2,last_index_,last_year_week,have_signed,last_sum_in_mln,date_signed,date_create_m,date_create_y
8975,15648,2023,12,20.12.2023 15:53:13,Ратникова Анна Владимировна,2023-12-20 12:53:13,Интерес,0.0,14.74,19823,2023-51,False,0.0,NaT,12,2023
8976,15649,2023,12,20.12.2023 16:13:56,Васильев Семен Вячеславович,2023-12-20 13:13:56,Интерес,0.0,90.80,19824,2023-51,False,0.0,NaT,12,2023
8977,15650,2023,12,21.12.2023 09:29:25,Ратникова Анна Владимировна,2023-12-21 06:29:25,Интерес,0.0,0.00,19825,2023-51,False,0.0,NaT,12,2023
8978,15651,2023,12,21.12.2023 10:30:01,Ратникова Анна Владимировна,2023-12-21 07:30:01,Интерес,0.0,0.00,34778,2023-51,False,0.0,NaT,12,2023
8979,15652,2023,12,21.12.2023 11:34:08,Ратникова Анна Владимировна,2023-12-21 08:34:08,Интерес,0.0,0.00,19827,2023-51,False,0.0,NaT,12,2023


In [295]:

# df = df_last_statuses_month.copy()
df = df.groupby(['date_last_state_y','date_last_state_m','last_stage_name']).agg(
    sum_in_m_amount = ('sum_in_mln','sum'),
    square_m2_amount =('square_m2','sum')    ,	 
    all_id  = ('order_id','unique'),
    count_id  = ('order_id','nunique'),
    order_manager = ('order_manager','last')

)
df[['sum_in_m_amount','square_m2_amount']] = df[['sum_in_m_amount','square_m2_amount']].astype('int')

df_group_week_states = df.reset_index('stage_name').sort_values(by=['year_week','stage_name'])
df_group_week_states.tail(5)

KeyError: 'date_last_state_y'

In [296]:

df = df_last_statuses.copy()
df = df.groupby(['year_week','stage_name']).agg(
    sum_in_m_amount = ('sum_in_mln','sum'),
    square_m2_amount =('square_m2','sum')    ,	 
    all_id  = ('order_id','unique'),
    count_id  = ('order_id','nunique'),
    order_manager = ('order_manager','last')

)
df[['sum_in_m_amount','square_m2_amount']] = df[['sum_in_m_amount','square_m2_amount']].astype('int')

df_group_week_states = df.reset_index('stage_name').sort_values(by=['year_week','stage_name'])
df_group_week_states

,stage_name,sum_in_m_amount,square_m2_amount,all_id,count_id,order_manager
year_week,,,,,,
2017-10,Договор подписан,40,1859,[6768],1,Павлюков Андрей
2017-10,Отказ,99,4007,"[6767, 6769, 6770, 6771, 6772, 6773, 6774]",7,Павлюков Андрей
2017-11,Отказ,215,10899,"[6775, 6776, 6777, 6778, 6779, 6780]",6,Дубовой Сергей
2017-12,Отказ,208,7346,"[6781, 6782, 6783, 6784, 6785, 6786, 6787]",7,Павлюков Андрей
2017-13,Договор подписан,22,871,[6794],1,Дубовой Сергей
...,...,...,...,...,...,...
2023-8,Неопределенность длительная,84,5194,"[13883, 13886, 13889, 13895]",4,Байкалов Александр Александрович
2023-8,Отказ,76,6909,"[13879, 13880, 13881, 13882, 13884, 13885, 13887, 13888, 13891, 13892, 13893, 13894, 13896, 13897, 13898, 13899, 139...",18,Дубовой Сергей
2023-9,Другая заявка,0,135,[13903],1,Самохина Анастасия


In [297]:
df_group_week_states.stage_name.value_counts()
df_group_week_states

,stage_name,sum_in_m_amount,square_m2_amount,all_id,count_id,order_manager
year_week,,,,,,
2017-10,Договор подписан,40,1859,[6768],1,Павлюков Андрей
2017-10,Отказ,99,4007,"[6767, 6769, 6770, 6771, 6772, 6773, 6774]",7,Павлюков Андрей
2017-11,Отказ,215,10899,"[6775, 6776, 6777, 6778, 6779, 6780]",6,Дубовой Сергей
2017-12,Отказ,208,7346,"[6781, 6782, 6783, 6784, 6785, 6786, 6787]",7,Павлюков Андрей
2017-13,Договор подписан,22,871,[6794],1,Дубовой Сергей
...,...,...,...,...,...,...
2023-8,Неопределенность длительная,84,5194,"[13883, 13886, 13889, 13895]",4,Байкалов Александр Александрович
2023-8,Отказ,76,6909,"[13879, 13880, 13881, 13882, 13884, 13885, 13887, 13888, 13891, 13892, 13893, 13894, 13896, 13897, 13898, 13899, 139...",18,Дубовой Сергей
2023-9,Другая заявка,0,135,[13903],1,Самохина Анастасия


In [58]:
def get_by_state(query_state: str):
    df = df_group_week_states.copy()
    # query_state = '045. Договорная стадия'

    df = df.query('stage_name == @query_state').copy()
    df['all_id_preview'] = df['all_id'].shift(1)

    df['value_plus'] = df.apply(lambda row: list(set(row['all_id']) - set(row['all_id_preview'])) if row['all_id_preview'] is not None else [], axis=1)
    df['value_minus'] = df.apply(lambda row: list(set(row['all_id_preview']) - set(row['all_id'])) if row['all_id_preview'] is not None else [], axis=1)

    df['dif_money'] = df['sum_in_m_amount'] - df['sum_in_m_amount'].shift(1)

    translata = {
        'stage_epoches': 'стадия',
        'sum_in_m_amount': 'сумма_всего',
        'square_m2_amount': "м2 всего",
        "all_id": "номера заявок",
        "count_id": "количество заявок",
        "all_id_preview": "н/з неделю назад",
        "value_minus": "н/з ушедшие",
        "value_plus": "н/з новые",
        "dif_money": "разница в млн"
    }
    return df.rename(columns=(translata)).tail(12)


In [59]:
get_by_state('Договор подписан').tail(6)
get_by_state('Договорная стадия').tail(6)
get_by_state('Договор подписан').tail(6)

,stage_name,сумма_всего,м2 всего,номера заявок,количество заявок,order_manager,н/з неделю назад,н/з новые,н/з ушедшие,разница в млн
year_week,,,,,,,,,,
2023-44,Договор подписан,75,2060,[15385],1,Шуляк Мария,[15302],[15385],[15302],55.0
2023-48,Договор подписан,98,2410,[15521],1,Шуляк Мария,[15385],[15521],[15385],23.0
2023-5,Договор подписан,46,824,[13789],1,Дубовой Сергей,[15521],[13789],[15521],-52.0
2023-6,Договор подписан,120,1030,[13830],1,Самохина Анастасия,[13789],[13830],[13789],74.0
2023-7,Договор подписан,67,868,[13856],1,Шуляк Мария,[13830],[13856],[13830],-53.0
2023-8,Договор подписан,49,1139,[13890],1,Дубовой Сергей,[13856],[13890],[13856],-18.0


In [60]:
df_group_week_states.stage_name.value_counts()

stage_name
Отказ                          356
Неопределенность длительная    237
Договор подписан               178
Другая заявка                  152
В работе                        97
Неопределенность короткая       58
Интерес                         26
Договорная стадия                6
Name: count, dtype: int64

# every order

In [63]:
df = df_last_statuses[['order_id','datetime_order_state','order_manager','year_week','order_state','life_satage_number','stage_name','sum_in_mln','square_m2']]
df = df.sort_values(by=['year_week','order_state']) #,15302,14256
# .query('order_id == [14256,13062]')

df = df.groupby(['order_id','stage_name']).agg(
    first_year_w_state = ('year_week','min'),
    last_year_w__state = ('year_week','max'),
    first_time_state = ('datetime_order_state','min'),
    last_time_state = ('datetime_order_state','max'),
    sum_mln = ('sum_in_mln','last'),	
    square_m2 = ('square_m2','last'),
    order_manager = ('order_manager','last'))
 
df.groupby(["order_id"]).agg('last').reset_index()
# df['dif_dasy_state'] = round((df['last_time_state']- df['first_time_state']) / np.timedelta64(1, 'D')).astype('int')
df
# (last=('last_time_state','last'))
df.reset_index() \
    .groupby(['order_id']) \
        .agg(last_time_state=('last_time_state','last')).reset_index() \
            .merge(
                df.reset_index()[['last_time_state',"order_id","sum_mln","square_m2","order_manager"]].reset_index(),
                on=['order_id','last_time_state'],how='inner'
                )

,order_id,last_time_state,index,sum_mln,square_m2,order_manager
0,6673,2017-05-24 06:51:47,0,22.4400,997.00,Дубовой Сергей
1,6674,2017-05-04 14:43:56,1,2.8485,144.00,Самохина Анастасия
2,6675,2017-03-03 12:06:05,2,105.8600,5581.00,Дубовой Сергей
3,6676,2017-02-15 09:32:58,3,22.9350,1092.00,Самохина Анастасия
4,6677,2017-08-01 13:19:05,4,2.0070,88.00,Полякова Анна
...,...,...,...,...,...,...
8975,15648,2023-12-20 12:53:13,8975,0.0000,14.74,Ратникова Анна Владимировна
8976,15649,2023-12-20 13:13:56,8976,0.0000,90.80,Васильев Семен Вячеславович
8977,15650,2023-12-21 06:29:25,8977,0.0000,0.00,Ратникова Анна Владимировна
8978,15651,2023-12-21 07:30:01,8978,0.0000,0.00,Ратникова Анна Владимировна


In [64]:
## strange order
df.sort_values(by='first_year_w_state').reset_index() \
    .rename(columns=({
        'order_id':'Номер заявки',
        'stage_name':'Этап',
        'first_year_w_state':'первое год-неделя-',
        'last_year_w__state':'крайняя год-неделя',
        "first_time_state":"первое повление время",
        'last_time_state':'крайнее появление время',
        'sum_mln':'сумма в млн',
        'square_m2':'м2',
        'order_manager':'Менеджер заявки',
    }))

,Номер заявки,Этап,первое год-неделя-,крайняя год-неделя,первое повление время,крайнее появление время,сумма в млн,м2,Менеджер заявки
0,6767,Отказ,2017-10,2017-10,2017-04-28 11:26:36,2017-04-28 11:26:36,78.620,2346.00,Дубовой Сергей
1,6774,Отказ,2017-10,2017-10,2017-03-10 14:44:15,2017-03-10 14:44:15,0.000,323.00,Павлюков Андрей
2,6773,Отказ,2017-10,2017-10,2017-03-10 13:09:28,2017-03-10 13:09:28,0.000,88.00,Павлюков Андрей
3,6772,Отказ,2017-10,2017-10,2017-03-10 13:09:50,2017-03-10 13:09:50,0.000,44.00,Павлюков Андрей
4,6768,Договор подписан,2017-10,2017-10,2017-09-18 05:08:53,2017-09-18 05:08:53,40.800,1859.00,Павлюков Андрей
...,...,...,...,...,...,...,...,...,...
8975,13915,Отказ,2023-9,2023-9,2023-03-02 11:42:37,2023-03-02 11:42:37,0.000,47.70,Лесин Александр
8976,13916,Отказ,2023-9,2023-9,2023-04-07 12:06:29,2023-04-07 12:06:29,63.828,1786.76,Байкалов Александр Александрович
8977,13917,Отказ,2023-9,2023-9,2023-03-06 06:33:33,2023-03-06 06:33:33,0.000,1925.08,Байкалов Александр Александрович
8978,13909,Отказ,2023-9,2023-9,2023-03-15 06:34:50,2023-03-15 06:34:50,0.000,66.00,Лесин Александр


In [65]:
df_last_statuses.groupby(['order_id','stage_name']).agg('count').nlargest(5,columns='order_state')

,,year_week,order_state,order_manager,life_satage_number,sum_in_mln,square_m2,order_status_year_create,order_date_week_number_state,datetime_order_state,_index_
order_id,stage_name,,,,,,,,,,
6673,Отказ,1,1,1,1,1,1,1,1,1,1
6674,Договор подписан,1,1,1,1,1,1,1,1,1,1
6675,Отказ,1,1,1,1,1,1,1,1,1,1
6676,Отказ,1,1,1,1,1,1,1,1,1,1
6677,Отказ,1,1,1,1,1,1,1,1,1,1


In [48]:
df_last_statuses.stage_name.value_counts().sort_index()
# .to_clipboard()

stage_name
В работе                        213
Договор подписан                267
Договорная стадия                 6
Другая заявка                   265
Интерес                          61
Неопределенность длительная    1134
Неопределенность короткая       136
Отказ                          6881
Name: count, dtype: int64

In [49]:
df_last_statuses.query('stage_name == "Другая заявка"').groupby(['order_id','stage_name']).agg('count').nlargest(5,columns='order_state')

,,year_week,order_state,order_manager,life_satage_number,sum_in_mln,square_m2,order_status_year_create,order_date_week_number_state,datetime_order_state,_index_
order_id,stage_name,,,,,,,,,,
7327,Другая заявка,1,1,1,1,1,1,1,1,1,1
7940,Другая заявка,1,1,1,1,1,1,1,1,1,1
8099,Другая заявка,1,1,1,1,1,1,1,1,1,1
8192,Другая заявка,1,1,1,1,1,1,1,1,1,1
8256,Другая заявка,1,1,1,1,1,1,1,1,1,1


In [50]:
df_analyse.query('order_id == 13062').to_clipboard()

## an

In [51]:
df = df_states[['order_id','order_manager','contact_person','note','object_delivery_date','order_supplier',]]
df.query('order_id == @list_another_order').to_clipboard()
# df_states.query('order_id == 13062').to_clipboard()

In [117]:
df_states.columns
df_process.columns

Index(['order_create_year_create', 'order_date_week_number_create', 'order_id', 'customer', 'date_order_create', 'square_m2', 'amount_sum', 'order_supplier', 'order_manager', 'AO', 'AC', 'order_status_year_create', 'order_date_week_number_state', 'date_time_state', 'state_type', 'order_state', 'order_state_check', 'last_first_sate', 'variaty', 'contact_initiator', 'contact_person', 'object_delivery_date', 'signing_date', 'note', 'note_author', 'stage_epoches', 'order_link', ' ',
       'order_state_time', 'life_satage_number', 'stage_name', 'datetime_order_create', 'datetime_order_state', 'year_week', '_index_', 'sum_in_mln'],
      dtype='object')

## df_clients

In [66]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)
df_state_rows # 'order_supplier'
# [['customer','order_id','order_supplier']]
df_all =  df_process.copy()

df_customer_unique = df_all.groupby(['customer','order_id'])\
    .agg(
        order_supplier_first = ('order_supplier','first'),


        order_create_year_first = ('order_create_year_create','first'),
        date_order_create = ('date_order_create','first'),
        order_id_first = ('order_id','first'),
        
        year_week_first = ('year_week','min'),
        year_week_last = ('year_week','max'),
        order_managers = ('order_manager','unique')
         )

df = df_customer_unique.reset_index()


df_customers = df_state_rows[['order_id','order_manager_last','have_signed','last_sum_in_mln','last_square_m2','date_signed']] \
    .merge(
        df[['order_id','customer','order_supplier_first','order_create_year_first','date_order_create','order_id_first','year_week_last','year_week_first']],
        on='order_id',how='outer'
    )



df_customers


,order_id,order_manager_last,have_signed,last_sum_in_mln,last_square_m2,date_signed,customer,order_supplier_first,order_create_year_first,date_order_create,order_id_first,year_week_last,year_week_first
0,6673,Дубовой Сергей,False,22.440,997.00,NaT,Транснефть,Эл. почта,2017,Дата создания завявки,6673,2017-2,2017-2
1,6674,Самохина Анастасия,True,2.848,144.00,2017-05-04 14:43:56,Таманьнефтегаз,Эл. почта,2017,10.01.2017 10:02:42,6674,2017-2,2017-2
2,6675,Дубовой Сергей,False,105.860,5581.00,NaT,UNEX STROY,Эл. почта,2017,11.01.2017 10:32:46,6675,2017-2,2017-2
3,6676,Самохина Анастасия,False,22.935,1092.00,NaT,СтройПроектМонтаж №7,Эл. почта,2017,11.01.2017 11:27:05,6676,2017-2,2017-2
4,6677,Полякова Анна,False,2.007,88.00,NaT,Автобан ДСК,Эл. почта,2017,12.01.2017 15:26:05,6677,2017-2,2017-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8976,15648,Ратникова Анна Владимировна,False,0.000,14.74,NaT,ДАКС-ИНЖИНИРИНГ,Эл. почта,2023,20.12.2023 15:53:13,15648,2023-51,2023-51
8977,15649,Васильев Семен Вячеславович,False,0.000,90.80,NaT,Лимак-констракшн,Эл. почта,2023,20.12.2023 16:13:56,15649,2023-51,2023-51
8978,15650,Ратникова Анна Владимировна,False,0.000,0.00,NaT,Всероссийский дважды ордена Трудового Красного Знамени Теплотехнический научно-исследовательский институт,Эл. почта,2023,21.12.2023 09:29:25,15650,2023-51,2023-51
8979,15651,Ратникова Анна Владимировна,False,0.000,0.00,NaT,Стройтехностандарт,Эл. почта,2023,21.12.2023 10:30:01,15651,2023-51,2023-51


In [279]:
df_customers

,order_id,order_manager_last,have_signed,last_sum_in_mln,last_square_m2,date_signed,customer,order_supplier_first,order_create_year_first,date_order_create,order_id_first,year_week_last,year_week_first
0,6673,Дубовой Сергей,False,22.440,997.00,NaT,Транснефть,Эл. почта,2017,Дата создания завявки,6673,2017-2,2017-2
1,6674,Самохина Анастасия,True,2.848,144.00,2017-05-04 14:43:56,Таманьнефтегаз,Эл. почта,2017,10.01.2017 10:02:42,6674,2017-2,2017-2
2,6675,Дубовой Сергей,False,105.860,5581.00,NaT,UNEX STROY,Эл. почта,2017,11.01.2017 10:32:46,6675,2017-2,2017-2
3,6676,Самохина Анастасия,False,22.935,1092.00,NaT,СтройПроектМонтаж №7,Эл. почта,2017,11.01.2017 11:27:05,6676,2017-2,2017-2
4,6677,Полякова Анна,False,2.007,88.00,NaT,Автобан ДСК,Эл. почта,2017,12.01.2017 15:26:05,6677,2017-2,2017-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8889,15561,Ратникова Анна Владимировна,False,0.000,368.00,NaT,Гипроцветмет,Эл. почта,2023,04.12.2023 13:50:03,15561,2023-49,2023-49
8890,15562,Ратникова Анна Владимировна,False,0.000,14.28,NaT,Политехстрой,Эл. почта,2023,04.12.2023 15:58:18,15562,2023-49,2023-49
8891,15563,Новиков Александр Валерьевич,False,0.000,0.00,NaT,СДС-Строй,Эл. почта,2023,04.12.2023 16:12:30,15563,2023-49,2023-49
8892,15564,Самохина Анастасия,False,0.000,39.20,NaT,ПАРКНЕФТЬ,Эл. почта,2023,04.12.2023 16:27:21,15564,2023-49,2023-49


In [67]:
df_customers_signed = df_customers.query('have_signed == True') \
    .groupby('customer').agg(
        sum_mln = ("last_sum_in_mln",'sum'),
        last_square_m2 = ("last_square_m2",'sum'),
        order_id_count = ('order_id','count'),
        order_create_year_first = ('order_create_year_first','first'),
        order_create_year_last = ('order_create_year_first','max'),
        date_order_create = ('date_order_create','first'),
        year_week_first = ('year_week_first','min'),
        year_week_last = ('year_week_last','max'),
        ).reset_index()
# df_customers_signed[['sum_mln','last_square_m2']] = df[['sum_mln','last_square_m2']].round(1)
df_customers_signed.tail()


,customer,sum_mln,last_square_m2,order_id_count,order_create_year_first,order_create_year_last,date_order_create,year_week_first,year_week_last
151,Электроснабсбыт,18.149,399.00,1,2019,2019,2019-10-02 12:54:03,2019-40,2019-40
152,Эльга-Строй,829.363,11485.06,4,2021,2023,2021-09-28 14:36:38,2021-39,2023-4
153,ЭнергоСеть,413.541,5395.30,1,2023,2023,15.05.2023 14:39:51,2023-20,2023-20
154,Югтерминалпроект,0.900,36.00,1,2018,2018,10.01.2018 10:28:46,2018-2,2018-2
155,Яковлевский ГОК,34.659,1208.00,2,2018,2018,02.07.2018 12:32:00,2018-27,2018-52


In [68]:
df = df_customers.query('have_signed == False') \
    .groupby('customer').agg(
        sum_mln = ("last_sum_in_mln",'sum'),
        last_square_m2 = ("last_square_m2",'sum'),
        order_id_count = ('order_id','count'),
        order_create_year_first = ('order_create_year_first','first'),
        order_create_year_last = ('order_create_year_first','max'),
        date_order_create = ('date_order_create','first'),
        year_week_first = ('year_week_first','min'),
        year_week_last = ('year_week_last','max'),
        ).reset_index()
df[['sum_mln','last_square_m2']] = df[['sum_mln','last_square_m2']].round(1)
df.tail()

,customer,sum_mln,last_square_m2,order_id_count,order_create_year_first,order_create_year_last,date_order_create,year_week_first,year_week_last
5567,фирма ГЕРА,10.3,370.0,3,2019,2022,2019-06-18 13:44:15,2019-25,2022-7
5568,фирма Руслан,0.0,364.0,1,2019,2019,2019-06-11 11:43:08,2019-24,2019-24
5569,фирма ФРИКОН,0.0,11051.0,1,2019,2019,2019-05-20 15:51:47,2019-21,2019-21
5570,фонд развития Соловецкого архипелага,0.0,0.0,1,2018,2018,19.11.2018 17:23:31,2018-47,2018-47
5571,Қамқор Менеджмент,0.0,0.0,1,2022,2022,01.12.2022 10:44:13,2022-48,2022-48


In [69]:
df_first_time_customer = df_customers_signed.groupby('order_create_year_first') \
    .agg(
        count_customer=('customer','count'),)
df_first_time_customer

df_customers.query('have_signed == True').columns


Index(['order_id', 'order_manager_last', 'have_signed', 'last_sum_in_mln', 'last_square_m2', 'date_signed', 'customer', 'order_supplier_first', 'order_create_year_first', 'date_order_create', 'order_id_first', 'year_week_last', 'year_week_first'], dtype='object')

In [70]:
df = df_customers.query('have_signed == True')[['order_id', 'order_manager_last', 'have_signed',
'last_sum_in_mln','last_square_m2'	,'date_signed'	,'customer',	'order_supplier_first']]. merge(
    df_customers_signed[['customer', 'order_create_year_first','order_create_year_last','date_order_create','year_week_first']],
    on = 'customer'
)
df['sign_year'] = pd.to_datetime(df['date_signed']).dt.year
# df['sign_first_year'] = df['sign_year'] == df.order_create_year_first
df.groupby('order_create_year_first') \
    .agg(
        count_new_customers = ('order_create_year_first','count'),

    )
df.groupby(['order_create_year_first','customer']).head(1)
# df['first_year'] = df['sign_year'] == df.order_create_year_first
df.order_create_year_first
df_customers

,order_id,order_manager_last,have_signed,last_sum_in_mln,last_square_m2,date_signed,customer,order_supplier_first,order_create_year_first,date_order_create,order_id_first,year_week_last,year_week_first
0,6673,Дубовой Сергей,False,22.440,997.00,NaT,Транснефть,Эл. почта,2017,Дата создания завявки,6673,2017-2,2017-2
1,6674,Самохина Анастасия,True,2.848,144.00,2017-05-04 14:43:56,Таманьнефтегаз,Эл. почта,2017,10.01.2017 10:02:42,6674,2017-2,2017-2
2,6675,Дубовой Сергей,False,105.860,5581.00,NaT,UNEX STROY,Эл. почта,2017,11.01.2017 10:32:46,6675,2017-2,2017-2
3,6676,Самохина Анастасия,False,22.935,1092.00,NaT,СтройПроектМонтаж №7,Эл. почта,2017,11.01.2017 11:27:05,6676,2017-2,2017-2
4,6677,Полякова Анна,False,2.007,88.00,NaT,Автобан ДСК,Эл. почта,2017,12.01.2017 15:26:05,6677,2017-2,2017-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8976,15648,Ратникова Анна Владимировна,False,0.000,14.74,NaT,ДАКС-ИНЖИНИРИНГ,Эл. почта,2023,20.12.2023 15:53:13,15648,2023-51,2023-51
8977,15649,Васильев Семен Вячеславович,False,0.000,90.80,NaT,Лимак-констракшн,Эл. почта,2023,20.12.2023 16:13:56,15649,2023-51,2023-51
8978,15650,Ратникова Анна Владимировна,False,0.000,0.00,NaT,Всероссийский дважды ордена Трудового Красного Знамени Теплотехнический научно-исследовательский институт,Эл. почта,2023,21.12.2023 09:29:25,15650,2023-51,2023-51
8979,15651,Ратникова Анна Владимировна,False,0.000,0.00,NaT,Стройтехностандарт,Эл. почта,2023,21.12.2023 10:30:01,15651,2023-51,2023-51


In [71]:
# df

## ca_crm

In [298]:
ca_crm_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQStrlBeO2KCV2sWwjzsl6Fq5bEN2LwV0WfB4G9mA0feA7lwXXXzQ1Xra6CLGb9mv3uI8QYW4j33xxZ/pub?gid=1451311609&single=true&output=csv'
df_ca_crm_raw = pd.read_csv(ca_crm_link)

In [299]:
df_ca_crm = df_ca_crm_raw.drop(columns=['Unnamed: 0'])
df = df_ca_crm.merge(df_customers.query('have_signed == True'), on='order_id', how='outer')

In [300]:
df[['order_id','price_cd_norm','sum_amount_norm','last_sum_in_mln']]
df[['price_cd_norm_mln','sum_amount_norm_mln']] = round(df[['price_cd_norm','sum_amount_norm']]/1_000_000,1)
df.drop(columns=['price_cd_norm','sum_amount_norm'])
df[['price_cd_norm_mln','sum_amount_norm_mln','last_sum_in_mln']]

,price_cd_norm_mln,sum_amount_norm_mln,last_sum_in_mln
0,0.0,0.0,NaN
1,0.0,5.3,604.588
2,129.6,129.6,129.622
3,18.2,18.2,NaN
4,110.5,0.5,NaN
...,...,...,...
498,NaN,NaN,1.360
499,NaN,NaN,5.333
500,NaN,NaN,111.486
501,NaN,NaN,0.260


In [301]:
df_acomplish = df[['order_id','contract_doc','date_order_create','have_signed' ,'date_signed', 'order_manager_last', 'client','company_signer','lin','manager',
'price_cd_norm_mln'	,'sum_amount_norm_mln'	,'square_with_cor_norm'	,'last_square_m2',
'year_week_last','year_week_first']]

In [302]:
link_our_objects = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQKxzhnKadVktCPsGR-Q6wF3G70w7a9rroE8YxyWksDtbq-n9O0FwnNKu1lxT0kmQ/pub?output=csv'
df_our_projects_raw = pd.read_csv(link_our_objects)

In [313]:
df = df_acomplish
df['date_order_create_m'] = pd.to_datetime(df['date_order_create'],dayfirst=True,).dt.month

df['date_order_create_y'] = pd.to_datetime(df['date_order_create'],dayfirst=True,).dt.year
df['date_order_create_m'] = df['date_order_create_m'].fillna(0).astype('int').astype('str').str.zfill(2)
df['date_order_create_y'] = df['date_order_create_y'].fillna(0).astype('int').astype('str')
df['new_date'] = df.apply(lambda row: create_month(row['date_order_create_m'], row['date_order_create_y']), axis=1)
df.query('contract_doc == "2613-1"')
# df[[order_id	contract_doc	date_order_create	have_signed	date_signed	price_cd_norm_mln	sum_amount_norm_mln	square_with_cor_norm date_order_create_m	date_order_create_y	new_date]]


,order_id,contract_doc,date_order_create,have_signed,date_signed,order_manager_last,client,company_signer,lin,manager,price_cd_norm_mln,sum_amount_norm_mln,square_with_cor_norm,last_square_m2,year_week_last,year_week_first,date_order_create_m,date_order_create_y,new_date
182,13419,2613-1,NaN,NaN,NaT,NaN,АнтрацитИнвестПроект,АнтрацитИнвестПроект,7.706461e+09,Байкалов Александр,4.1,4.1,0.0,NaN,NaN,NaN,00,0,01.00.0


In [ ]:
[[Номер ДД - корр.	Тип ДД	Номер ОД	Статус ДД	Дата заключения ДД	Клиент CRM	Номер заявки CRM, породивщей ДД	Завод-Подписант Cтоимость итого c НДС из ДД, руб.]]

In [304]:



translate = {'Название заказчика':'company_name',"Адрес поставки":'adres',
             '№ договора НАШ!':'contract_doc'}
loaded_dict.update(translate)


df = df_our_projects_raw.rename(columns=loaded_dict) \
    .drop(columns=['Самовывоз',	'Аренда','Подтверждено по 1С','Завод',"Примечение"])
result_series = df[df['adres'].notnull()].adres.agg('unique')

# # result_df = pd.DataFrame({'adres': result_series})
# # result_df = result_df.reset_index(drop=True)
# df = df.groupby(['company_name']) \
#     .agg(
#         comp_name = ('INN','unique'),
#         adres = ('adres','nunique'),
#         adres_uni = ('adres','unique'),
#         contract_our = ('contract_our','unique')
#     ).reset_index()
# df['_index_'] = df.index # 391
# df_normal = df.query('adres ==1')

# df.query('adres ==2').to_clipboard()

In [64]:
df[['contract_doc','Unnamed: 5','adres']].merge(df_acomplish,on='contract_doc',how='inner').to_clipboard()

In [306]:
df

,company_name,INN,Год заключения договора,contract_doc,Unnamed: 5,Тип здания,adres,Есть на карте Объекты РПК «Модуль»?
0,"ООО ""ЮнионКомпани""",7.708561e+09,2006,71,https://drive.google.com/file/d/1UgdVgVRumLZq9uYZpVqHkx4yLjQVPqXO/view?usp=sharing,NaN,"г.Москва,ММДЦ ""Москва-Сити"", Краснопресненская набережная, участок №9",NaN
1,"ООО ""ЮнионКомпани""",7.708561e+09,2006,70К,https://drive.google.com/file/d/1JSPE_fKMomyKeb8UZl7AadMeB5qbLWrQ/view?usp=sharing,NaN,г.Юхнов Калужской обл.,70К
2,"ООО ""ЮнионКомпани""",7.708561e+09,2006,66,https://drive.google.com/file/d/1RifvRxlWmzxS8EuSVACSFNaNhc_hwBxk/view?usp=sharing,NaN,NaN,NaN
3,"ООО ""ЮнионКомпани""",7.708561e+09,2006,64,https://drive.google.com/file/d/1A4ls2_e7XtL3DVAnh2Oxvhfr-afuN7zc/view?usp=sharing,NaN,г.Домодедово Московской области,NaN
4,"ООО ""ЮнионКомпани""",7.708561e+09,2006,63,https://drive.google.com/file/d/1ppGE3n3f3Tcfr-HXwIcVKgaw5eCQ99mK/view?usp=sharing,NaN,г.Домодедово Московской области,NaN
...,...,...,...,...,...,...,...,...
1240,"ООО ""Выксунский металлургический завод""",NaN,2021,2515-1,NaN,NaN,"г.Выкса, Нижегородская область",NaN
1241,ООО Сфера,NaN,2022,2554,NaN,NaN,"г. Санкт-Петербург муницип. округ Гагаринское, пр. Юрия Гагарина",NaN
1242,ООО Сфера,NaN,2022,2546,NaN,NaN,"г. Санкт-Петербург муницип. округ Гагаринское, пр. Юрия Гагарина д.8",NaN
1243,ООО Стеллар Констракшн,NaN,2022,2552,NaN,NaN,"Нижегородская область г.Выкса, Проммикрорайон - 7",NaN


In [305]:
# реестр конкуренты

link_resstr_conc ='https://docs.google.com/spreadsheets/d/e/2PACX-1vQQqOAyJsjkRWhRXbZuIQyZUamcfdcyH8gw93nV_cv-Ca0dEa3yKbVpFqEBW2hqVjJhcHm_uj1DVfzM/pub?output=xlsx'
df_reestr_compet_raw = pd.read_excel(link_resstr_conc)

In [311]:
# df_reestr_compet_raw

In [67]:
df_ = df_reestr_compet_raw[[
    'Вид процедуры','Реестровый номер закупки','Дата публикации','Заказчик',
    'ИНН Заказчика','Регион','Наименование объекта','Площадь','Этажность',
    'Количество','Цена за ед.'
]]

translate_dict = {
    'Наименование объекта':'object_name','Дата публикации':'date_pub','Заказчик':'client','Регион':'region'
}
# \n65 262 500 ₽
df_ = df_[df_["Реестровый номер закупки"].notnull()]

df_['norm_price'] = df_['Цена за ед.'] \
    .astype('str') \
    .str.strip() \
    .str.replace(' ','') \
    .str.replace('₽','') \
    .str.replace('(,*)','',regex=True) \
    .str.replace('-','') \
    .str.replace('Неустановлена','0') \
    .str.replace(' ','') \
    .fillna('0')
df_['square'] = df_['Площадь']\
    .astype('str') \
    .str.strip() \
    .str.replace(' ','') \
    .str.replace('(,*)','',regex=True) \
    .str.replace('м2','',) \
    .fillna(0)
df_['number'] = df_['Количество'].astype('str').str.extract('(\d*)').fillna('0').apply(pd.to_numeric, errors='coerce')
df_['registry_numb'] = df_['Реестровый номер закупки'].astype('str')
df_.rename(columns=translate_dict)[['object_name','date_pub','client','region','norm_price','square','number','registry_numb']].to_clipboard()
# df_[df_["norm_price"].notn()]["norm_price"]
# # df_[['Количество','norm_price']].agg('sum')
